In [2]:
import torch
import catboost
from torch import nn
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import TensorDataset,DataLoader

In [3]:
def preprocessing_Y(file_path):
    Y = pd.read_csv(file_path).iloc[:,:]
    Y = Y[Y.Filename != 'train_01046']
    enc = OneHotEncoder().fit(Y[['Label']])
    Y_one_hot = enc.transform(Y[['Label']]).toarray()
    Y_one_hot = torch.FloatTensor(Y_one_hot)
    print('Y_ont_hot shape',Y_one_hot.shape)
    print('Y_df shape',Y.shape)
    return Y_one_hot,Y

def load_pretrain_senet(model_path):
    model = torch.hub.load(
        'moskomule/senet.pytorch',
        'se_resnet20',
        num_classes=6)
    model.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model = torch.nn.DataParallel(model, device_ids=[0, 1, 2 ,3])
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def get_senet_output(senet,data):
    return senet(data)

def get_all_senet_output(data):
    y1 = senet1(data)
    y2 = senet2(data)
    y3 = senet3(data)
    y4 = senet4(data)
    y5 = senet5(data)
    y6 = senet6(data)
    combined = torch.cat([y1,y2,y3,y4,y5,y6],dim=-1)
    return combined.detach().cpu().numpy()

def get_X_numpy(X):
    X_train_np = np.array([[]])
    data_iter = DataLoader(TensorDataset(torch.FloatTensor(X.to(torch.float32))),batch_size=256)
    for bx in tqdm(data_iter):
        bx = bx[0]
        bx = bx.to('cuda:0')
        y_hat = get_all_senet_output(bx)
        if len(X_train_np) == 1:
            X_train_np = y_hat
        else:
            X_train_np = np.vstack((X_train_np,y_hat))
    return X_train_np

# 路徑

In [4]:
Y_train_path = 'train/meta_train.csv'
senet1 = 'senet20_2021_0604_0750.pt'
senet2 = 'senet20_2021_0604_1312_randomseed168.pt'
senet3 = 'senet20_2021_0604_1345_randomseed210.pt'
senet4 = 'senet20_wu_add_data_seed84.pt'
senet5 = 'senet20_wu_add_data_seed126.pt'
senet6 = 'senet20_wu_add_data_seed210.pt'

# load k 個 senet

In [5]:
senet1 = load_pretrain_senet(senet1).to('cuda:0')
senet2 = load_pretrain_senet(senet2).to('cuda:0')
senet3 = load_pretrain_senet(senet3).to('cuda:0')
senet4 = load_pretrain_senet(senet4).to('cuda:0')
senet5 = load_pretrain_senet(senet5).to('cuda:0')
senet6 = load_pretrain_senet(senet6).to('cuda:0')

Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master
Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master
Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master
Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master
Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master
Using cache found in /root/.cache/torch/hub/moskomule_senet.pytorch_master


# 訓練 和 測試資料

In [6]:
X_train_all = torch.load('X_train_吳啟聖教的方法.pt')
X_test_all = torch.load('X_test_吳啟聖教的方法.pt')
Y_train_all,Y_train_df = preprocessing_Y(Y_train_path)

Y_ont_hot shape torch.Size([1199, 6])
Y_df shape (1199, 3)


# train k個 catboost model

In [7]:
from sklearn.model_selection import train_test_split
i = 0
catboost_model = {}
for seed in tqdm([42,42*2,42*3,42*4,42*5,42*6]):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, Y_train_all, 
                                                          test_size=0.2, 
                                                          random_state=seed,
                                                          stratify=Y_train_all)
    X_train_np = get_X_numpy(X_train)
    y_train_np = y_train.detach().numpy().argmax(axis=1)
    X_valid_np = get_X_numpy(X_valid)
    y_valid_np = y_valid.detach().numpy().argmax(axis=1)
    model = CatBoostClassifier(iterations=250,verbose=False)
    model.fit(X_train_np,y_train_np,use_best_model=True,eval_set=(X_valid_np,y_valid_np))
    catboost_model[seed] = model.copy()
    print(i,model.score(X_valid_np,y_valid_np))
    gc.collect()
    i +=1

100%|██████████| 4/4 [00:12<00:00,  3.04s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

0 0.9375



100%|██████████| 4/4 [00:00<00:00,  5.22it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

1 0.9666666666666667



100%|██████████| 4/4 [00:00<00:00,  5.35it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

3 0.9541666666666667



100%|██████████| 4/4 [00:00<00:00,  5.15it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

4 0.9458333333333333



100%|██████████| 4/4 [00:00<00:00,  5.41it/s]

100%|██████████| 6/6 [01:01<00:00, 10.20s/it]

5 0.975


In [9]:
X_test_np = get_X_numpy(X_test_all)
X_test_np.shape

100%|██████████| 40/40 [00:07<00:00,  5.06it/s]


(10000, 36)

In [10]:
p1 = catboost_model[42*1].predict_proba(X_test_np)
p2 = catboost_model[42*2].predict_proba(X_test_np)
p3 = catboost_model[42*3].predict_proba(X_test_np)
p4 = catboost_model[42*4].predict_proba(X_test_np)
p5 = catboost_model[42*5].predict_proba(X_test_np)
p6 = catboost_model[42*6].predict_proba(X_test_np)

In [11]:
final_prob = (p1+p2+p3+p4+p5+p6) / 6
final_prob

array([[1.02085748e-02, 1.15241286e-02, 2.90965893e-02, 2.16360995e-02,
        1.31107412e-01, 7.96427196e-01],
       [9.24101329e-04, 8.29499687e-04, 1.12103605e-03, 1.81482715e-03,
        2.05725749e-03, 9.93253278e-01],
       [9.81195851e-01, 2.04034460e-03, 3.17417318e-03, 5.13248465e-03,
        6.04385933e-03, 2.41328760e-03],
       ...,
       [1.78117311e-02, 8.97464319e-01, 2.81212492e-02, 1.87979262e-02,
        1.23113755e-02, 2.54933993e-02],
       [2.61030668e-02, 1.29246836e-02, 8.95621743e-01, 1.84297471e-02,
        1.72061319e-02, 2.97146274e-02],
       [4.73992747e-02, 8.30568005e-03, 9.23694484e-01, 5.52986472e-03,
        6.96289790e-03, 8.10779902e-03]])

In [12]:
final_prob.sum(axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [13]:
sample_submit = pd.read_csv('sample_submission.csv')
sample_submit.iloc[:10000,1:] = final_prob
sample_submit

,Filename,Barking,Howling,Crying,COSmoke,GlassBreaking,Other
0,public_00001,0.010209,0.011524,0.029097,0.021636,0.131107,0.796427
1,public_00002,0.000924,0.000829,0.001121,0.001815,0.002057,0.993253
2,public_00003,0.981196,0.002040,0.003174,0.005132,0.006044,0.002413
3,public_00004,0.014720,0.010538,0.020623,0.052997,0.051967,0.849155
4,public_00005,0.933125,0.006343,0.046743,0.005772,0.004100,0.003918
...,...,...,...,...,...,...,...
29995,private_19996,0.166600,0.166600,0.166600,0.166600,0.166600,0.167000
29996,private_19997,0.166600,0.166600,0.166600,0.166600,0.166600,0.167000
29997,private_19998,0.166600,0.166600,0.166600,0.166600,0.166600,0.167000
29998,private_19999,0.166600,0.166600,0.166600,0.166600,0.166600,0.167000


In [14]:
# save
sample_submit.to_csv('stacking_model_submit.csv',index=False)
print('done')

done
